In [ ]:
import logging
from pathlib import Path

import hvplot.xarray  # noqa: F401
import rioxarray  # noqa: F401
from darts_preprocessing.data_sources.planet import load_planet_scene
from dask.distributed import Client
from odc.stac import configure_rio
from rich import traceback
from rich.logging import RichHandler

from darts.utils.logging import setup_logging

setup_logging()
logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)
traceback.install(show_locals=False)
client = Client()
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)
client

In [ ]:
DATA_ROOT = Path("../data")

# fpath = DATA_ROOT / "input/planet/PSOrthoTile/4372514/5790392_4372514_2022-07-16_2459"
fpath = DATA_ROOT / "input/planet/PSOrthoTile/4974017/5854937_4974017_2022-08-14_2475"
arcticdem_dir = DATA_ROOT / "download/arcticdem"

In [ ]:
# load planet scene
ds_planet = load_planet_scene(fpath).isel(x=slice(0, 2000), y=slice(6000, 8000))
ds_planet

## Get the Tile

In [ ]:
from darts_acquisition.arcticdem.datacube import get_arcticdem_tile

ds = get_arcticdem_tile(ds_planet, arcticdem_dir, resolution=2, buffer=0)
ds

In [ ]:
crs = ds.rio.crs.to_string()
ds.dem.hvplot.image(aggregator="max", rasterize=True, cmap="terrain", crs=crs, projection=crs)

In [ ]:
crs = ds_planet.rio.crs.to_string()
dem_plot = ds.dem.rio.reproject_match(ds_planet).hvplot.image(aggregator="max", rasterize=True, cmap="terrain")
red_plot = ds_planet.red.hvplot.image(x="x", y="y", aggregator="mean", rasterize=True, cmap="reds")
dem_plot + red_plot

## Calculate Relative Elevation and Slope

In [ ]:
from darts_preprocessing.engineering.arcticdem import calculate_slope, calculate_topographic_position_index

ds = calculate_topographic_position_index(ds)
ds = calculate_slope(ds)
ds

In [ ]:
ds = ds.persist()

In [ ]:
crs = ds.rio.crs.to_string()
dem_plot = ds.dem.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, cmap="terrain", crs=crs, projection=crs, title="DEM"
)
tpi_plot = ds.tpi.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, cmap="terrain", crs=crs, projection=crs, title="TPI"
)
slope_plot = ds.slope.hvplot.image(
    x="x", y="y", aggregator="max", rasterize=True, cmap="terrain", crs=crs, projection=crs, title="Slope"
)
dem_plot + tpi_plot + slope_plot